Bringing together GDP and Personal Income by US County

In [3]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
from sklearn import decomposition
import statsmodels.regression.linear_model as lm
from sklearn.ensemble import RandomForestRegressor 
from typing import List, Tuple, Dict

%run ./mobilityHelpers.ipynb

MYDIR = "./../../ResearchProposal/"

myFiles = os.listdir(MYDIR)

gdpFile = MYDIR + "bea_gov/gdp/gdp_ready_to_analyze.csv"
piFile = MYDIR + "bea_gov/personal_income/personal_income_ready_to_analyze.csv"
hhiFile = MYDIR + "income_inequality/census_income_by_county/hh_income__census_data.csv"
populationFile = MYDIR + "population_dynamics/census_population_data_2010_2019.csv"
suicideFile = MYDIR + "suicide/multiple_causes_of_death__suicide.csv"
employmentFile = MYDIR + "unemployment/employment_by_county_state_year.csv"
stateAbbrevFile = MYDIR + "state_abbreviations.csv"
myfiles = {"gdp": gdpFile,
           "pi": piFile,
           "hhi": hhiFile,
           "pop": populationFile,
           "sc": suicideFile,
           "emp": employmentFile,
          }

/Users/agilgur/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [5]:
class mobilityMLhelpers:
    def __init__(self, 
                 myFiles: Dict[str, str] = myfiles,
                 ctv_cutoff: float = 0.055):
        self.mCTVcutoff = ctv_cutoff
        self.mFiles = myfiles
        
        self.mStateAbbreviationsDF = pd.read_csv(stateAbbrevFile)
       
    def splitDataIntoTrainTest(self, myPCAdata: pd.DataFrame) -> Tuple[float]:
        y = myPCAdata["NETMIG"].copy()
        X = myPCAdata[[cc for cc in myPCAdata.columns if "NETMIG" not in cc]].copy()

        # Splitting the dataset into the Training set and Test set
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)
        
        return X_train, X_test, y_train, y_test
    
    
    """
    Fit RFR given the n_est; test the regressor by computing its RSq 
    on unseen data and plotting RFR prediction with unseen data
    """
    def fitRFR(self, n_est, X_train, y_train, X_test, y_test, showplots: bool = True):
        myRFR = RandomForestRegressor(n_estimators=n_est)
        myRFR.fit(X_train, y_train)
        myRFR.feature_importances_

        y_predict_test = myRFR.predict(X=X_test)

        error_term = [list(y_test)[ii] - list(y_predict_test)[ii] for ii in range(len(list(y_test)))]
        var_error = np.var(error_term)
        RSq = 1.0 - (var_error) / np.var(y_test)

        print(f"""RSq = {RSq: .4f}""")
        if showplots:
            fig, ax = plt.subplots(figsize=(10, 8))
            plt.scatter(y_test, y_predict_test)
            ax.set_xlabel("y_test")
            ax.set_ylabel("RFR(" + str(n_est) + ") predict")
            ax.set_title("RFR with " + str(n_est) + " estimators")
            plt.show()

        return(RSq, myRFR)


    """
    Run PCA; cut off PCs with explained_variance_ >= min_explained_var
    """
    def runPCA(self, min_explained_var, myPCAdata, verbose: bool = True):
        
        X_train, X_test, y_train, y_test = \
            self.splitDataIntoTrainTest(myPCAdata)
        
        pca = PCA()
        X_train_pca = pca.fit_transform(X_train)
        X_test_pca = pca.transform(X_test)
        
        if verbose:
            print("------------PCA Explained Variance------------")
            print(pca.explained_variance_)
            print("------------PCA Explained Variance Ratio------------")
            print(pca.explained_variance_ratio_)
            
        """
        Rerun PCA for n_components = number of PCs with 
        explained_variance_ >= min_explained_var
        """
        
        ppc = [ee for ee in list(pca.explained_variance_ratio_) if ee > EXPL_VAR]
        if verbose:
            print(f"""------------Components with Explained Variance Ratio > {EXPL_VAR} ------------""")
            print(len(ppc))

        pca = PCA(n_components=len(ppc))
        X_train = pca.fit_transform(X_train)
        X_test = pca.transform(X_test)
        if verbose:
            print("------------PCA Explained Variance------------")
            print(pca.explained_variance_)
            print("------------PCA Explained Variance Ratio------------")
            print(pca.explained_variance_ratio_)
    
        return pca, X_train, X_test, y_train, y_test
    
    
    
    """
    Details in 
    https://stackoverflow.com/questions/31909945/obtain-eigen-values-and-vectors-from-sklearn-pca/31941631#31941631
    
    Apply PCA; select the primary principal components:
    (a) outliers in eigenvalues (default);
    (b) PCs with eigenvalues greater than a given tolerance % of max eigenvalue; 
    (c) or  with the lowest frequency on the histogram.
    
    Note: the outlier-based selection will only work if we have more than 10-15 PCs.  
    This method is sensitive to number of PCs.
    """
    def applyPCA(self, 
                 gdpiData: pd.DataFrame,
                 cols: List[str],
                 n_comps: int,
                 identify_outliers: bool = True,
                 beta_outliers: float = 1.5,
                 tolerance: float = -1.0,
                 remove_most_frequent: bool = False,
                 
                 verbose: bool = False,
                 showplots: bool = False) -> Tuple[pd.DataFrame, List[float]]:


        data = myDF[cols]

        if verbose:
            print(f"""We have {len(cols)} population-dynamics variables """
              f"""to use for modeling net migration into each county""")

            print(f"""We are going to reduce it by identifying the"""
                  f"""primary components.  We will start with {n_comps}""")
        
        """Normalize by StDev:"""
        data /= np.std(data, axis=0)

        pcaMdl = decomposition.PCA(n_components=n_comps)
        pcaMdl.fit(data)

        if verbose:
            print("Eigenvectors:")
            print(pcaMdl.components_)
            print("Eigenvalues:")
            print(pcaMdl.explained_variance_)

        eigenvalues = pcaMdl.explained_variance_
        transformed = pcaMdl.transform(data)

        for cc in range(n_comps):
            myDF["pc_" + str(cc)] = [transformed[ii][cc] for ii in range(len(transformed))]

        """Identify Outliers in Eigenvalues"""
        if identify_outliers:
            print(f"""Of these {n_comps}, we will select """
                  f"""the EigenValue outliers, to avoid overfitting the model""")
            ax = plt.subplot()
            ax.boxplot(eigenvalues, vert=False)
            ax.set_title("Eigenvalues of the principal components", fontsize=14)
            plt.show()
            
            """
            Select primary components as outliers in Eigenvalues
            """
            qrtls = np.percentile(eigenvalues, q=(25.0, 75.0))
            qrtls
            iqr = max(qrtls) - min(qrtls)
            high_bound = max(qrtls) + beta_outliers * iqr
            high_bound

            primary_components = [ee for ee in eigenvalues if ee >= high_bound]
            print(f"""In this case, primary components are ones """
                  f"""with eigenvalues >= {high_bound}""")
        else:
            primary_components = [ee for ee in eigenvalues if ee >= max(qrtls)]
            print(f"""In this case, primary components are ones """
                  f"""with eigenvalues >= {max(qrtls)}""")
             
        print(f"""We have {len(primary_components)} primary principal components:""")
        for ii in range(len(primary_components)):
            print(f"""{ii}: {primary_components[ii]:.3f}""")
            
        if tolerance > 0:
            maxPC = max(eigenvalues)
            print(f"""Removing PCs with eigenvalues < {(100.0*tolerance): .3f}% of {maxPC}""")
            primary_components = [pc for pc in eigenvalues if pc >= tolerance * maxPC]
            
        if remove_most_frequent:
            ax = plt.subplot()
            myHist = ax.hist(eigenvalues)
            if verbose:
                print(myHist[0])
                print(myHist[1])
            counts = list(myHist[0])
            maxCountIndex = counts.index(max(counts))
            maxCountEigenvalue = myHist[1][maxCountIndex]
            
            primary_components = [pp for pp in eigenvalues if pp > maxCountEigenvalue]
            
        return myDF, list(primary_components)
    
    """
    Apply linear regression
    """
    def applyLinRegr(self, 
                     pcPopDF: pd.DataFrame, 
                     year: int,
                     primary_components: List[float],
                     verbose: bool = True,
                     showplots: bool = True,
                    ) -> pd.DataFrame:
        print(year)

        yVar = "NETMIG" + str(year)
        frmla = yVar + " ~ "
        frmla

        rhs = " + ".join(["pc_" + str(ii) for ii in range (len(primary_components))])
        rhs

        frmla += rhs
        frmla
        
        linMdl = lm.OLS.from_formula(formula=frmla, data = pcPopDF)
        res = linMdl.fit()

        if verbose:
            print(res.summary())
        
        pcPopDF[yVar + "_LM"] = res.predict()
        
        if showplots:
            ax = pcPopDF.plot.scatter(x=yVar, 
                                 y=yVar + "_LM", 
                                 figsize=(10, 6))
            ax.set_title(yVar + " and linear regression prediction",
                         fontsize=16
                        )
            ax.set_xlabel(yVar, fontsize=14)
            ax.set_ylabel(yVar + " model prediction", fontsize=14)
            plt.show()
            
        return pcPopDF
    
    """
    Fit random forest regression to get the importance of each of the primary components
    """
    def fitRFRdf(self, 
               pcPopDF: pd.DataFrame, 
               xVars: List[str] = [],
               yVarBase: str = "NETMIG",
               year: int = 2010,
               ctv_cutoff: float = 0.055,
               n_rfr_trees: int = 100,
               verbose: bool = False,
              ) -> Tuple[List[str], pd.DataFrame]:
        
        # create regressor object 
        regressor = RandomForestRegressor(n_estimators=n_rfr_trees, random_state = 0) 

        # fit the regressor with x and y data 
        if yVarBase is None or len(yVarBase) == 0:
            yVarBase = "NETMIG"
            
        yVar = yVarBase + str(year)
        
        x = pcPopDF[xVars]
        y = pcPopDF[yVar]
        rfr = regressor.fit(x, y) 
    
        pcPopDF[yVar + "_rfr_predict"] = regressor.predict(pcPopDF[xVars])
        importances = list(rfr.feature_importances_)
        importances

        important_features = \
            [ii for ii in range(len(importances)) if importances[ii] > ctv_cutoff]

        importantXs = [xVars[impfeat] for impfeat in important_features]
        
        if verbose:
            print(f"""Feature Importances (contributions to variance):""")

            """sort xVars in order of importances"""
            ctvDict = {} # Dict[str: int]
            for ii in range(len(importances)):
                ctvDict[xVars[ii]] = importances[ii]
                
            ctvlist = sorted(ctvDict.items(), key=lambda x: x[1], reverse=True)
            for kk in ctvlist:
                print(f"""{kk[0]}: {kk[1]: .3f}""")
                
            print(f"""Identified important features (CTV cutoff = {ctv_cutoff})\n""")

            
            for kk in ctvlist:
                if kk[0] in importantXs:
                    print(f"""{kk[0]}: {kk[1]: .3f}""")
                    
            print(f"""\nRFR R^2 with identified features: """
                  f"""{sum([importances[ii] for ii in important_features]): .3f}""")

#         importantXs = ["pc_" + str(ii) for ii in important_features]
        
    
        return [xVars[impfeat] for impfeat in important_features], pcPopDF
    
    def applyLinearRegression(self,
                              pcPopDF: pd.DataFrame,
                              importantXs: List[str],
                              yVarBase: str = "NETMIG",
                              year: int = 2010,
                              verbose: bool = True,
                              showplots: bool = True,
                             ) -> pd.DataFrame:
        
        yVar = yVarBase + str(year)
        frmla = yVar + " ~ "
        print(frmla)

        rhs = " + ".join(importantXs)
        print(rhs)
        

        frmla += rhs
        print(frmla)

        verbose=True
        showplots=True

        linMdl = lm.OLS.from_formula(formula=frmla, data = pcPopDF)
        res = linMdl.fit()

        if verbose:
            print(res.summary())

        pcPopDF[yVar + "_LM"] = res.predict()

        if showplots:
            ax = pcPopDF.plot.scatter(x=yVar, 
                                 y=yVar + "_LM", 
                                 figsize=(10, 6))
            ax.set_title(yVar + " and linear regression prediction",
                         fontsize=16
                        )
            ax.set_xlabel(yVar, fontsize=14)
            ax.set_ylabel(yVar + " model prediction", fontsize=14)
            plt.show()

    def plot_importances(self, forest, X, y):
        plt.rcParams.update({"font.size": 16})
        importances = forest.feature_importances_
        std = np.std([tree.feature_importances_ for tree in forest.estimators_],
                     axis=0)
        indices = np.argsort(-1 * importances)

        # Plot the feature importances of the forest
        plt.figure(figsize=(15, 8))
        plt.title("Feature importances")
        plt.bar(range(X.shape[1]), importances[indices],
               color="r", yerr=std[indices], align="center")
        # If you want to define your own labels,
        # change indices to a list of labels on the following line.
        plt.xticks(range(X.shape[1]), indices)
        plt.xlim([-1, X.shape[1]])
        plt.show()  
        
    def plot_explained_variance_pca(self, pca):
        plt.rcParams.update({"font.size": 16})
        
        plt.figure(figsize=(15, 8))
        plt.title("Explained Variance Ratio")
        plt.bar(range(len(list(pca.explained_variance_ratio_))), 
                pca.explained_variance_ratio_,
               color="r", align="center")
        plt.show()